## ¿Podemos establecer alguna relación lineal entre la productividad de los ABFs y su categoría, banca y precios del banco?

Este cuaderno busca encontrar una relación lineal entre la productividad de los ABFs en términos de su banca, categoría y precios a los que se les solicita vender.

In [1]:
import pandas as pd

In [2]:
df = pd.read_excel("../../db/datos_productividad.xlsx", sheet_name="BT")
df["TPP"] = df["TPP"] * 100

In [3]:
# quiero hacer una regresion productividad como combinacion lineal de tipo categoria y precio actual. 
# Posiblemente también mes y año
df.head(3)

,AÑO,MES,AÑO-MES,FEC_SALDO,ABF.ASIGNADOS,ABF.PRODUCTIVOS,Cantidad_A,Cantidad_B,Cantidad_C,Desembolso A,Desembolso B,Desembolso C,Cantidad creditos A,Cantidad creditos B,Cantidad creditos C,Ticket promedio A,Ticket promedio B,Ticket promedio C,TPP
0,2022,ene,ene-2022,44592,181,152,16,45,91,690614.241875,464523.244889,280651.508901,8,6,3,120568.369733,97730.631693,117573.132374,17.28
1,2022,feb,feb-2022,44620,187,162,24,56,82,910432.094583,492368.947679,318803.086098,10,5,3,121002.772049,124817.339238,114325.478170,17.00
2,2022,mar,mar-2022,44651,182,164,23,54,87,860410.628261,640832.940182,320865.761954,11,7,4,100515.071473,128193.887678,92428.674683,17.32


In [4]:
cols = ["AÑO", "MES", "Cantidad_A", "Cantidad_B", "Cantidad_C", "Desembolso A", "Desembolso B",
        "Desembolso C", "TPP"]

df[cols].head(3)

,AÑO,MES,Cantidad_A,Cantidad_B,Cantidad_C,Desembolso A,Desembolso B,Desembolso C,TPP
0,2022,ene,16,45,91,690614.241875,464523.244889,280651.508901,17.28
1,2022,feb,24,56,82,910432.094583,492368.947679,318803.086098,17.00
2,2022,mar,23,54,87,860410.628261,640832.940182,320865.761954,17.32


In [5]:
dic_A = {"Categoría": ["A"] * len(df), "Año": df["AÑO"].to_list(), "Mes": df["MES"].tolist(), 
         "Cantidad ABFs": df["Cantidad_A"].to_list(), "Desembolso": df["Desembolso A"].tolist(),
         "TPP": df["TPP"].tolist()}

dic_B = {"Categoría": ["B"] * len(df), "Año": df["AÑO"].to_list(), "Mes": df["MES"].tolist(), 
         "Cantidad ABFs": df["Cantidad_B"].to_list(), "Desembolso": df["Desembolso B"].tolist(),
         "TPP": df["TPP"].tolist()}

dic_C = {"Categoría": ["C"] * len(df), "Año": df["AÑO"].to_list(), "Mes": df["MES"].tolist(), 
         "Cantidad ABFs": df["Cantidad_A"].to_list(), "Desembolso": df["Desembolso C"].tolist(),
         "TPP": df["TPP"].tolist()}

df_A = pd.DataFrame.from_dict(dic_A)
df_B = pd.DataFrame.from_dict(dic_B)
df_C = pd.DataFrame.from_dict(dic_C)


df_final = pd.concat([df_A, df_B, df_C])
df_final["Desembolso"] = df_final["Desembolso"].astype("int64")
df_final.head(3)

,Categoría,Año,Mes,Cantidad ABFs,Desembolso,TPP
0,A,2022,ene,16,690614,17.28
1,A,2022,feb,24,910432,17.00
2,A,2022,mar,23,860410,17.32


## Correr modelo lineal simple por MMCO

El objetivo inicial de la regresión era establecer una relación lienal entre el desembolso de los ABFs según el precio al que deben vender, la categoría a la que pertenecen, el año y mes en el que se está vendiendo, así como la cantidad de ABFs activos. Por esa razón, la regresión que observaremos no contempla un análisis de selección de parámetros ni balanceo entre módulo de coeficientes y suma de errores al cuadrado. Solamente se corre el MMCO y se establecen conclusiones con ello.

In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
import numpy as np

In [7]:
df_final = pd.get_dummies(df_final, columns=["Categoría"], drop_first=True)
df_final = pd.get_dummies(df_final, columns=["Año"], drop_first=True)
df_final = pd.get_dummies(df_final, columns=["Mes"], drop_first=True)

In [8]:
df_final.columns

Index(['Cantidad ABFs', 'Desembolso', 'TPP', 'Categoría_B', 'Categoría_C',
       'Año_2023', 'Año_2024', 'Mes_ago', 'Mes_dic', 'Mes_ene', 'Mes_feb',
       'Mes_jul', 'Mes_jun', 'Mes_mar', 'Mes_may', 'Mes_nov', 'Mes_oct',
       'Mes_sep'],
      dtype='object')

In [9]:
cols = ["Categoría_B", "Categoría_C", 'Año_2023', 'Año_2024', 'Mes_ago',
        'Mes_dic', 'Mes_ene', 'Mes_feb', 'Mes_jul', 'Mes_jun', 'Mes_mar',
        'Mes_may', 'Mes_nov', 'Mes_oct', 'Mes_sep']

for col in cols:
    df_final[col] = df_final[col].astype(int)

In [10]:
X_entrenamiento, X_testeo, Y_entrenamiento, Y_testeo = train_test_split( df_final[["Categoría_B", "Categoría_C", 'Año_2023', 'Año_2024', 'Mes_ago',
        'Mes_dic', 'Mes_ene', 'Mes_feb', 'Mes_jul', 'Mes_jun', 'Mes_mar',
        'Mes_may', 'Mes_nov', 'Mes_oct', 'Mes_sep', "TPP"]],
                            df_final["Desembolso"],
                            test_size=0.2, random_state=42)


In [11]:
X_entrenamiento_intercepto = sm.add_constant(X_entrenamiento)

modelo = sm.OLS(Y_entrenamiento, X_entrenamiento_intercepto).fit()
print(modelo.summary())

                            OLS Regression Results                            
Dep. Variable:             Desembolso   R-squared:                       0.877
Model:                            OLS   Adj. R-squared:                  0.845
Method:                 Least Squares   F-statistic:                     27.66
Date:                Wed, 16 Oct 2024   Prob (F-statistic):           3.09e-22
Time:                        16:58:23   Log-Likelihood:                -1006.0
No. Observations:                  79   AIC:                             2046.
Df Residuals:                      62   BIC:                             2086.
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const        1.714e+06    2.6e+05      6.602      

Conclusiones: 
    - Solamente el mes de marzo tiene un coeficiente significativo en la regresión. 
    - La categoría si es significativa para el desembolso de un ABF
    - La TPP de venta es significativa para el desembolso de un ABF

## Calibración del modelo

In [12]:
X_ajustada = X_entrenamiento[["Categoría_B", "Categoría_C", "TPP"]]
X_ajustada_intercepto = sm.add_constant(X_ajustada)

modelo = sm.OLS(Y_entrenamiento, X_ajustada_intercepto).fit()
print(modelo.summary())

                            OLS Regression Results                            
Dep. Variable:             Desembolso   R-squared:                       0.767
Model:                            OLS   Adj. R-squared:                  0.757
Method:                 Least Squares   F-statistic:                     82.10
Date:                Wed, 16 Oct 2024   Prob (F-statistic):           1.24e-23
Time:                        16:58:23   Log-Likelihood:                -1031.3
No. Observations:                  79   AIC:                             2071.
Df Residuals:                      75   BIC:                             2080.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const        1.277e+06   1.16e+05     10.979      

Al eliminar la variable de tiempo, vemos que el coeficiente de TPP deja de ser estadísticamente significativo. Por tanto, la TPP influye de acuerdo al momento en el tiempo en el que ocurre.

In [13]:
X_ajustada = X_entrenamiento[["Categoría_B", "Categoría_C", "TPP",'Mes_mar']]
X_ajustada_intercepto = sm.add_constant(X_ajustada)

modelo = sm.OLS(Y_entrenamiento, X_ajustada_intercepto).fit()
print(modelo.summary()) 

                            OLS Regression Results                            
Dep. Variable:             Desembolso   R-squared:                       0.787
Model:                            OLS   Adj. R-squared:                  0.775
Method:                 Least Squares   F-statistic:                     68.29
Date:                Wed, 16 Oct 2024   Prob (F-statistic):           4.37e-24
Time:                        16:58:23   Log-Likelihood:                -1027.7
No. Observations:                  79   AIC:                             2065.
Df Residuals:                      74   BIC:                             2077.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const        1.242e+06   1.13e+05     11.016      

In [14]:
df_final.iloc[df_final["Desembolso"].idxmax()]

Cantidad ABFs         59.00
Desembolso       1221904.00
TPP                   16.83
Categoría_B            0.00
Categoría_C            0.00
Año_2023               1.00
Año_2024               0.00
Mes_ago                0.00
Mes_dic                0.00
Mes_ene                0.00
Mes_feb                0.00
Mes_jul                0.00
Mes_jun                0.00
Mes_mar                1.00
Mes_may                0.00
Mes_nov                0.00
Mes_oct                0.00
Mes_sep                0.00
Name: 14, dtype: float64